In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Navigate to the directory where your file is located
import os
os.chdir('/content/drive/MyDrive/Data MSc thesis')

# Read the CSV file
data = pd.read_csv("Preprocessed_15min_2023.csv")

In [ ]:
import pandas as pd

# Assuming 'year' is a column in your DataFrame 'data'
data_year_encoded = pd.get_dummies(data, columns=['year'])

In [ ]:
import pandas as pd

# Assuming 'month' is a column in your DataFrame 'data_year_encoded'
# Map the month numbers to month names
month_names = {1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June',
               7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'}

# Map month numbers to month names and then one-hot encode
data_month_encoded = pd.get_dummies(data_year_encoded, columns=['month'])
data_month_encoded.rename(columns=month_names, inplace=True)

In [ ]:
data_day_encoded = pd.get_dummies(data_month_encoded, columns=['day'])
#data_month_encoded.rename(columns=month_names, inplace=True)

In [ ]:
data_dayofweek_encoded = pd.get_dummies(data_day_encoded, columns=['day_of_week'])
#data_month_encoded.rename(columns=month_names, inplace=True)

In [ ]:
# Assuming 'weekend' is the column containing integer values (0 and 1)
data_dayofweek_encoded['weekend'] = data_dayofweek_encoded['weekend'].astype(bool)
data_dayofweek_encoded['is_public_holiday'] = data_dayofweek_encoded['is_public_holiday'].astype(bool)

In [ ]:
data_hour_encoded = pd.get_dummies(data_dayofweek_encoded, columns=['hour'])
#data_month_encoded.rename(columns=month_names, inplace=True)

In [ ]:
data_minute_encoded = pd.get_dummies(data_hour_encoded, columns=['minute'])
#data_month_encoded.rename(columns=month_names, inplace=True)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Assuming 'intensiteit' is the column you want to standardize
intensiteit_values = data_minute_encoded['intensiteit'].values.reshape(-1, 1)

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler to the intensiteit values and transform them
intensiteit_scaled = scaler.fit_transform(intensiteit_values)

# Replace the 'intensiteit' column with the standardized values
data_minute_encoded['intensiteit'] = intensiteit_scaled

In [ ]:
# Sort the data by id_meetlocatie and date
data1 = data_minute_encoded.sort_values(by=["id_meetlocatie", "date"])

# Group by id_meetlocatie and add the lagged feature
data1["avg_speed_t-1"] = data1.groupby("id_meetlocatie")["avg_speed"].shift(1)

In [ ]:
# Sort the data by id_meetlocatie and date
data2 = data1.sort_values(by=["id_meetlocatie", "date"])

# Group by id_meetlocatie and add the lagged feature
data2["avg_speed_t-2"] = data2.groupby("id_meetlocatie")["avg_speed_t-1"].shift(1)

In [ ]:
# Sort the data by id_meetlocatie and date
data3 = data2.sort_values(by=["id_meetlocatie", "date"])

# Group by id_meetlocatie and add the lagged feature
data3["avg_speed_t-3"] = data3.groupby("id_meetlocatie")["avg_speed_t-2"].shift(1)

In [ ]:
data4 = data3.dropna(subset=['avg_speed_t-1', 'avg_speed_t-2', 'avg_speed_t-3'])

In [ ]:
from sklearn.preprocessing import StandardScaler

# Columns to standardize
columns_to_standardize = ['avg_speed_t-1', 'avg_speed_t-2', 'avg_speed_t-3']

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler to the selected columns and transform them
data4[columns_to_standardize] = scaler.fit_transform(data4[columns_to_standardize])

<ipython-input-17-94757408da41>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data4[columns_to_standardize] = scaler.fit_transform(data4[columns_to_standardize])


In [ ]:
encoded_data = pd.get_dummies(data4, columns=['id_meetlocatie'])

In [ ]:
encoded_data.shape

(765552, 110)

In [ ]:
# vanaf 15 maart 2019 test split
# 19 oktober 2018 val split
# tot 19 oktober 2018 train split

In [ ]:
import pandas as pd

# Assuming 'data' is your DataFrame containing the dataset

# Convert 'date' column to datetime type
encoded_data['date'] = pd.to_datetime(encoded_data['date'])

# Perform train-validation-test split based on date
train_data = encoded_data[encoded_data['date'] < '2023-09-13']
test_data = encoded_data[encoded_data['date'] > '2023-10-19']
val_data = encoded_data[(encoded_data['date'] >= '2023-09-13') & (encoded_data['date'] <= '2023-10-19')]

# Optionally, you may reset the index of each split DataFrame
train_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)
val_data.reset_index(drop=True, inplace=True)

In [ ]:
train_data = train_data.drop(columns=['date'])
val_data = val_data.drop(columns=['date'])
test_data = test_data.drop(columns=['date'])

In [ ]:
print(train_data.shape)
print(val_data.shape)
print(test_data.shape)

(533892, 109)
(77528, 109)
(154132, 109)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np


# Extract features and target variable from train, validation, and test sets
X_train = train_data.drop(columns=['avg_speed'])
y_train = train_data['avg_speed']

X_val = val_data.drop(columns=['avg_speed'])
y_val = val_data['avg_speed']

X_test = test_data.drop(columns=['avg_speed'])
y_test = test_data['avg_speed']

#Define the parameter distributions for Random Forest hyperparameters
param_distributions = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 100, 200],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2 ,3],
    'max_features': ['log2', 'auto']
}

# Initialize Random Forest regressor
rf_regressor = RandomForestRegressor(random_state=42, n_jobs = -1)

# Perform randomized search for hyperparameter tuning
random_search = RandomizedSearchCV(
    rf_regressor,
    param_distributions=param_distributions,
    n_iter=3,
    cv=3,
    verbose=1,
    n_jobs=-1
)

# Fit the RandomizedSearchCV on the training data
random_search.fit(X_train, y_train)

# Get the best estimator and its hyperparameters
best_estimator = random_search.best_estimator_
best_params = random_search.best_params_

# Predict on the validation set using the best estimator
y_val_pred = best_estimator.predict(X_val)

# Calculate RMSE and MAE on the validation set
rmse_val = np.sqrt(mean_squared_error(y_val, y_val_pred))
mae_val = mean_absolute_error(y_val, y_val_pred)

print("Validation set metrics:")
print("RMSE:", rmse_val)
print("MAE:", mae_val)

# Predict on the test set using the best estimator
y_test_pred = best_estimator.predict(X_test)

# Calculate RMSE and MAE on the test set
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
mae_test = mean_absolute_error(y_test, y_test_pred)

print("\nTest set metrics:")
print("RMSE:", rmse_test)
print("MAE:", mae_test)

# Print the best hyperparameters
print("\nBest hyperparameters:")
print(best_params)

# Append predictions to validation and test data
#val_data['predictions_rf_15min'] = y_val_pred
test_data['predictions_rf_15min'] = y_test_pred

# Save predictions to new dataframes
#predictions_val_rf_15min = val_data.copy()
predictions_test_rf_15min = test_data.copy()

# Optionally, you can save these dataframes to CSV files
#predictions_val_rf_15min.to_csv('predictions_val_rf_15min_2023_new.csv', index=False)
predictions_test_rf_15min.to_csv('predictions_test_rf_15min_2023_new.csv', index=False)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


Validation set metrics:
RMSE: 4.219733276642439
MAE: 1.6725450986668735

Test set metrics:
RMSE: 4.015275051379388
MAE: 1.516073877333114

Best hyperparameters:
{'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 200}
